# This the task for GEN-AI from Vincent's Granvilles book "Synthetic Data and Generative AI"

The first thing is to import the packages

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy
from scipy.stats import ks_2samp
from statsmodels.distributions.empirical_distribution import ECDF






Reading the data from Github from the Vincent Granville's [repo](https://github.com/VincentGranville/Main) 

The link for the project book is [here](https://github.com/VincentGranville/Large-Language-Models/blob/main/Projects4.pdf)

In [34]:
# Selecting the needed data frames according to the project book. 
insurance = 'insurance.csv'
insurance_compare = 'insurance_compare.csv'

# Getting the URL
url = "https://raw.githubusercontent.com/VincentGranville/Main/main/"

# Separating the raw data frames
raw_insurance =  url + insurance 
raw_insurance_compare = url + insurance_compare 

# Opening the links with pandas.
df_insurance = pd.read_csv(raw_insurance)
df_insurance_compare = pd.read_csv(raw_insurance_compare)


### Step 1: Making the histograms for the numerical variables 

In [ ]:
fig, axs = plt.subplots(2, 2)
axs[0, 0].hist(df_insurance['age'])
axs[0, 0].set_title("Age")
axs[1, 0].hist(df_insurance['charges'])
axs[1, 0].set_title("Charges")
axs[0, 1].hist(df_insurance['bmi'])
axs[0, 1].set_title("Bmi")
fig.tight_layout()




## Step 2: perform evaluations of vendor comparisons



In [35]:
# Watch out with this code, since I am overwriting the dataset you need to run first the second cell or an error will be thrown. This can be fixed if you use a script. 
df_insurance_compare = df_insurance_compare.drop('region', axis=1) # drop columns that are not useful
df_insurance_compare = df_insurance_compare.dropna(axis='columns') # Drop na row wise. 

# Create a dataset of insurance compare with only Real category
df_insurance_compare_real = df_insurance_compare.loc[df_insurance_compare['Data'] == 'Real'] 

df_insurance_compare_real = df_insurance_compare_real.drop('Data', axis=1) # drop all the NA values from Data column
df_insurance_compare_real = df_insurance_compare_real.to_numpy()
df_insurance_compare_real


array([[1.900000e+01, 1.000000e+00, 2.790000e+01, 0.000000e+00,
        1.000000e+00, 1.688492e+04],
       [1.800000e+01, 0.000000e+00, 3.377000e+01, 1.000000e+00,
        0.000000e+00, 1.725550e+03],
       [2.800000e+01, 0.000000e+00, 3.300000e+01, 3.000000e+00,
        0.000000e+00, 4.449460e+03],
       ...,
       [1.800000e+01, 1.000000e+00, 3.685000e+01, 0.000000e+00,
        0.000000e+00, 1.629830e+03],
       [2.100000e+01, 1.000000e+00, 2.580000e+01, 0.000000e+00,
        0.000000e+00, 2.007950e+03],
       [6.100000e+01, 1.000000e+00, 2.907000e+01, 0.000000e+00,
        1.000000e+00, 2.914136e+04]])